# An Analysis

This notebook will be rendered on the website. Below is an example plot.


In [1]:
from policyengine_us import Simulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd
import numpy as np


def modify_parameters(parameters):
    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[
        0
    ].amount.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=1_000
    )

    parameters.gov.contrib.ubi_center.basic_income.amount.person.by_age[
        1
    ].threshold.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=17
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.rate.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=0.1
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.threshold.HEAD_OF_HOUSEHOLD.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=10_000
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.threshold.JOINT.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=20_000
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.threshold.SEPARATE.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=10_000
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.threshold.SINGLE.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=10_000
    )

    parameters.gov.contrib.ubi_center.basic_income.phase_out.threshold.WIDOW.update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=10_000
    )

    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)


In [8]:
EARNINGS_MAX = 55_000
EARNINGS_INCREMENT = 1_000
earnings_increments = int(EARNINGS_MAX / EARNINGS_INCREMENT) + 1

def make_household(married: bool, children: int):
    people = {
        "head": {
            "age": {2023: 30},
        },
    }

    households = {
        "household": {
            "members": ["head"],
            "state_code": {2023: "DC"},
        },
    }

    tax_units = {
        "tax_unit": {"members": ["head"], "premium_tax_credit": {2023: 0}}
    }

    spm_units = {"spm_unit": {"members": ["head"]}}

    if married:
        people["spouse"] = {"age": {2023: 30}}
        households["household"]["members"].append("spouse")
        tax_units["tax_unit"]["members"].append("spouse")
        spm_units["spm_unit"]["members"].append("spouse")

    for i in range(children):
        child_key = f"child_{i+1}"
        people[child_key] = {"age": {2023: 10}}
        households["household"]["members"].append(child_key)
        tax_units["tax_unit"]["members"].append(child_key)
        spm_units["spm_unit"]["members"].append(child_key)

    return {
        "people": people,
        "households": households,
        "tax_units": tax_units,
        "spm_units": spm_units,
        "axes": [
            [
                dict(
                    name="employment_income",
                    count=earnings_increments,
                    min=0,
                    max=EARNINGS_MAX,
                )
            ]
        ],
    }


def create_df(married: bool, children: int):
    hh = make_household(married, children)

    # Baseline
    baseline = Simulation(situation=hh)
    reformed = Simulation(situation=hh, reform=reform)
    return pd.DataFrame(
        dict(
            is_married="Married" if married else "Single",
            children=children,
            employment_income=baseline.calculate(
                "employment_income", 2023, map_to="household"
            ),
            net_income_baseline=baseline.calculate(
                "household_net_income", 2023
            ),
            net_income_reform=reformed.calculate("household_net_income", 2023),
        )
    )


l = []
for married in [False, True]:
    for children in range(1, 4):
        l.append(create_df(married, children))
df = pd.concat(l)


In [11]:
df["net_income_diff"] = df["net_income_reform"] - df["net_income_baseline"]

LIGHT_GRAY = "#F5F5F5"
GRAY = "#BDBDBD"
BLUE = "#5091cc"
LIGHT_BLUE = "lightblue"
DARK_BLUE = "darkblue"

COLOR_MAP = {0: GRAY, 1: LIGHT_BLUE, 2: BLUE, 3: DARK_BLUE}

LABELS = dict(
    employment_income="Adjusted gross income",
    married="Married",
    children="Children",
    net_income_diff="Financial Support for Families with Children",
)

import plotly.express as px

fig = px.line(
    df,
    "employment_income",
    "net_income_diff",
    color="children",
    facet_col="is_married",
    labels=LABELS,
    title="FSFC by marital status, number of children, and income",
    color_discrete_map=COLOR_MAP,
    facet_col_spacing=0.1,
)

fig.update_layout(
    xaxis_tickformat="$,",
    yaxis_tickformat="$,",
    plot_bgcolor="white",
    xaxis_gridcolor="white",
    yaxis_gridcolor="white",
)
fig.update_xaxes(tickformat="$,")

fig.for_each_annotation(lambda a: a.update(text=a.text.replace("is_married=", "")))
fig.show()


In [41]:
from policyengine_us import Microsimulation

sim = Microsimulation()
state = sim.calc("state_code", map_to="household")
# Count number in DC.
(state == "DC").values.sum()
ids = sim.calc("household_id", map_to="household")
ids.groupby(state).value_counts().loc["DC"].sum()

1135

In [40]:
ids.groupby(state).value_counts().sum()

59148